In [4]:
import os
import json
import re  # 정규표현식 라이브러리 추가
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 디버깅 설정 (오류 발생 지점 파악용)
# -----------------------------
# CUDA 오류가 발생하는 정확한 위치를 찾기 위해 동기화 모드를 활성화합니다.
# 이 줄은 코드의 맨 위에, torch를 import하기 전에 위치해야 합니다.
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# -----------------------------
# 1. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

# AutoProcessor를 사용하여 토크나이저와 이미지 프로세서를 한 번에 로드합니다.
processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16  # VRAM 사용량을 줄이기 위해 float16을 사용합니다.
)

# -----------------------------
# 2. OCR + 이미지 → JSON 변환 함수
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    
    # 최종 개선된 프롬프트
    prompt = (
        "<image>\n"
        "이 이미지를 참고하여 모든 약품 정보를 JSON 형식으로 추출하세요. "
        "JSON key는 '원료약품 및 그 분량', '성상', '효능·효과', '용법·용량' 등 이미지에 명시된 항목 이름을 사용하세요. "
        "각 key의 value에는 해당 항목 아래의 모든 텍스트를 포함시키세요. "
        "오직 JSON 객체만 반환하고 다른 설명은 절대 추가하지 마세요.\n"
        f"OCR 텍스트: {ocr_texts}"
    )
    
    # 입력 준비
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)
    
    # 모델 실행
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )
    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    
    # 정규표현식을 사용한 JSON 파싱 로직 개선
    try:
        # 응답 텍스트에서 첫 번째 유효한 JSON 객체 패턴을 찾습니다.
        json_match = re.search(r'\{[^{}]*\}', result_text)
        
        if json_match:
            json_string = json_match.group(0)
            return json.loads(json_string)
        else:
            print(f"⚠️ JSON 객체를 찾을 수 없습니다. 원문: {result_text}")
            return {"result_text": result_text}
            
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON 파싱 실패: {e}")
        return {"result_text": result_text}

# -----------------------------
# 3. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"C:\Potenup\Drug-Detection-Chatbot\data\medicine_00451.jpeg"
    
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")

📌 Using device: cuda


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


📌 OCR 추출 결과: ['[원료약품 및 그 분량]19 중', '·유효성분:', '디플루코르톨론발레레이트', '(BP) 3mg', '첨가제(보존제):', '파라옥시벤조산메틸(KP)', '1.8mg', '파라옥시벤조산프로필(KP)', '.0.2mg', '기타첨가제:', '경질유동파라핀,라우릴황산', '나트륨,모노스테아르산소르', '비탄,세탄올,스테아릴알코', '올,정제수,카보머940,트', '롤아민,프로필렌글리콜', '[성상]', '흰색~미담황색의 균질한 로션제', '[효능·효과]', '첨부문서참조', '[용법·용량]', '1일 2~3회 앞게  바른다.', '증상이 호전되면 1일 1회로', '충분하다.']


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


📌 LLM JSON 결과:
{
  "원료약품 및 그 분량": [
    "19 중",
    "유효성분:",
    "디플루코르톨론발레이트",
    "(BP) 3mg",
    "첨가제(보존제):",
    "파라옥시벤조산메틸(KP)",
    "1.8mg",
    "파라옥시벤조산프로필(KP)",
    ".0.2mg"
  ],
  "성상": [
    "흰색~미담황색의 균질한 로션제"
  ],
  "효능·효과": [
    "올,정제수,카보머940,트",
    "롤아민,프로필렌글리콜"
  ],
  "용법·용량": [
    "1일 2~3회 앞게  바른다.",
    "증상이 호전되면 1일 1회로"
  ]
}
✅ JSON 파일 저장 완료: output_medicine_00451.json


In [ ]:
import os
import json
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 디버깅 설정 (오류 발생 지점 파악용)
# -----------------------------
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# -----------------------------
# 1. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

# 모델을 직접 불러와 GPU에 로드합니다.
processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 2. OCR + 이미지 → JSON 변환 함수
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    try:
        # 이미지 파일이 존재하는지 확인
        if not os.path.exists(image_path):
            print(f"❌ Error: Image file not found at {image_path}")
            return {"error": "Image file not found."}
            
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return {"error": "Failed to load image."}
    
    # <image> 토큰을 프롬프트 시작 부분에 명시적으로 추가
    prompt = (
        "<image>"
        "아래 OCR 텍스트는 참고용일 뿐입니다. 이미지를 분석하여 다음 JSON 객체를 완성하세요. "
        "JSON key는 '원료약품 및 그 분량', '성상', '효능·효과', '용법·용량' 등 이미지에 명시된 항목 이름입니다. "
        "각 key의 value에는 해당 항목 아래의 모든 텍스트를 포함시켜야 합니다.\n"
        "오직 JSON 객체만 반환하고 다른 설명은 절대 추가하지 마세요.\n"
        f"OCR 텍스트: {ocr_texts}"
        "\n\n```json\n"
    )
    
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)

    # 이전에 inputs에 pixel_values가 누락되는 문제를 해결하기 위해 명시적으로 추가
    # inputs 딕셔너리에 'pixel_values'가 포함되지 않은 경우를 대비
    if 'pixel_values' not in inputs:
        # 이미지를 다시 처리하여 pixel_values를 추출
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        inputs['pixel_values'] = pixel_values

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7
    )
    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    
    try:
        json_matches = re.findall(r'```json\n([\s\S]*?)```', result_text)
        if json_matches:
            json_string = json_matches[-1]
            return json.loads(json_string)

        json_matches_fallback = re.findall(r'\{[\s\S]*\}', result_text)
        if json_matches_fallback:
            json_string_fallback = json_matches_fallback[-1]
            return json.loads(json_string_fallback)
        else:
            print(f"⚠️ JSON 객체를 찾을 수 없습니다. 원문: {result_text}")
            return {"result_text": result_text}
            
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON 파싱 실패: {e}")
        return {"result_text": result_text}

# -----------------------------
# 3. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = "/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00451.jpeg"
    
    try:
        from drugocr import extract_text
    except ImportError:
        print("⚠️ 'drugocr' 라이브러리를 찾을 수 없습니다. 임시 더미 함수를 사용합니다.")
        def extract_text(image_path):
            return [
                '[원료약품 및 그 분량]1g 중',
                '·유효성분:',
                '디플루코르톨론발레레이트',
                '(BP) 3mg',
                '첨가제(보존제):',
                '파라옥시벤조산메틸(KP)',
                '1.8mg',
                '파라옥시벤조산프로필(KP)',
                '0.2mg',
                '기타첨가제:',
                '경질유동파라핀, 라우릴황산',
                '나트륨, 모노스테아르산소르',
                '비탄, 세탄올, 스테아릴알코',
                '올, 정제수, 카보머940, 트',
                '롤아민, 프로필렌글리콜',
                '[성상]',
                '흰색~미담황색의 균질한 로션제',
                '[효능·효과]',
                '첨부문서참조',
                '[용법·용량]',
                '1일 2~3회 얇게 바른다.',
                '증상이 호전되면 1일 1회로',
                '충분하다.'
            ]
            
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    if 'error' not in json_result:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(json_result, f, ensure_ascii=False, indent=2)
        print(f"✅ JSON 파일 저장 완료: {output_file}")
    else:
        print(f"❌ JSON 파일 저장 실패. 오류: {json_result['error']}")

In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. 키 패턴 정의 (일반화)
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'·?유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]']
}

# -----------------------------
# 2. OCR 후처리 (일반화)
# -----------------------------
def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        # 키 탐지
        matched = False
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, line):
                    if buffer and current_key:
                        data[current_key] = ' '.join(buffer).replace('  ', ' ')
                    buffer = []
                    current_key = key
                    # 패턴 제거 후 buffer에 남은 텍스트 저장
                    line_clean = re.sub(pattern, '', line).strip()
                    if line_clean:
                        buffer.append(line_clean)
                    matched = True
                    break
            if matched:
                break
        # 키가 아니면 현재 buffer에 추가
        if current_key and not matched and line:
            buffer.append(line)

    # 마지막 buffer 처리
    if buffer and current_key:
        data[current_key] = ' '.join(buffer).replace('  ', ' ')

    return data

# -----------------------------
# 3. JSON 블록 추출
# -----------------------------
def extract_json_from_text(text):
    text_clean = re.sub(r'```json|```', '', text, flags=re.IGNORECASE).strip()
    matches = re.findall(r'\{.*?\}', text_clean, flags=re.DOTALL)
    if matches:
        json_string = matches[-1].replace("'", '"').strip()
        return json_string
    else:
        return None

# -----------------------------
# 4. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_general(ocr_texts)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "JSON key는 다음과 같습니다: "
        + ', '.join(KEY_PATTERNS.keys()) + ". "
        "값이 없으면 null로 표시하고, 광고/제조사 등 약 정보와 관련 없는 내용은 무시하세요.\n"
        f"OCR 항목별 텍스트:\n{preprocessed_ocr}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    json_string = extract_json_from_text(result_text)

    if json_string:
        try:
            data = json5.loads(json_string)
            return filter_json_fields(data)
        except Exception as e:
            print(f"⚠️ JSON 파싱 실패: {e}")
            return {"result_text": result_text}
    else:
        print("⚠️ JSON 블록을 찾을 수 없습니다.")
        return {"result_text": result_text}

# -----------------------------
# 5. 후처리: 키 필터 + null 처리
# -----------------------------
def filter_json_fields(data):
    return {k: data.get(k, None) for k in KEY_PATTERNS.keys()}

# -----------------------------
# 6. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # LLM JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. 키 패턴 정의 (일반화)
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'·?유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]']
}

# -----------------------------
# 2. OCR 후처리 (일반화)
# -----------------------------
def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    text_combined = ' '.join(ocr_texts).strip()

    # 띄어쓰기 및 오타 보정
    text_combined = re.sub(r'(\S)(,)(\S)', r'\1, \3', text_combined)
    text_combined = re.sub(r'(\S)(\()', r'\1 (', text_combined)
    text_combined = re.sub(r'(\))(\S)', r') \2', text_combined)
    text_combined = text_combined.replace("앞게", "얇게")
    text_combined = text_combined.replace("·유효성분:", " 유효성분: ")
    text_combined = text_combined.replace("●첨가제", " 첨가제")
    
    # 정규표현식을 사용하여 모든 키 패턴을 하나의 문자열로 결합
    all_patterns = '|'.join([p for patterns in KEY_PATTERNS.values() for p in patterns])
    
    # 텍스트를 각 키를 기준으로 분리
    parts = re.split(f'({all_patterns})', text_combined, flags=re.IGNORECASE | re.DOTALL)
    
    current_key_name = None
    for part in parts:
        if not part or part.strip() == '':
            continue
        
        # 키 패턴에 매칭되는지 확인
        matched = False
        for key_name, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, part, re.IGNORECASE):
                    current_key_name = key_name
                    # 키 패턴 이후의 텍스트를 현재 값으로 설정
                    value_text = re.sub(pattern, '', part).strip()
                    if value_text:
                        if data[current_key_name]:
                            data[current_key_name] += ' ' + value_text
                        else:
                            data[current_key_name] = value_text
                    matched = True
                    break
            if matched:
                break
        
        # 키 패턴에 매칭되지 않으면 이전 키의 값에 추가
        if not matched and current_key_name:
            if data[current_key_name]:
                data[current_key_name] += ' ' + part.strip()
            else:
                data[current_key_name] = part.strip()

    # '원료약품 및 그 분량'에 다른 키들이 포함되어 있다면 분리
    if data['원료약품 및 그 분량']:
        temp_data = data['원료약품 및 그 분량']
        for key, value in data.items():
            if key in ['유효성분', '첨가제(보존제)', '첨가제(동물유래성분)', '기타첨가제'] and value:
                temp_data = temp_data.replace(value, '').strip()
        data['원료약품 및 그 분량'] = temp_data

    return data


# -----------------------------
# 3. JSON 블록 추출
# -----------------------------
def extract_json_from_text(text):
    text_clean = re.sub(r'```json|```', '', text, flags=re.IGNORECASE).strip()
    matches = re.findall(r'\{.*?\}', text_clean, flags=re.DOTALL)
    if matches:
        json_string = matches[-1].replace("'", '"').strip()
        return json_string
    else:
        return None

# -----------------------------
# 4. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_general(ocr_texts)

    # 딕셔너리를 문자열로 변환하여 프롬프트에 포함
    ocr_text_str = json.dumps(preprocessed_ocr, ensure_ascii=False, indent=2)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "JSON key는 '원료약품 및 그 분량', '유효성분', '첨가제(보존제)', '첨가제(동물유래성분)', '기타첨가제', '성상', '효능·효과', '용법·용량'입니다. "
        "값이 없으면 null로 표시하고, 광고/제조사 등 약 정보와 관련 없는 내용은 무시하세요.\n"
        f"OCR 항목별 텍스트:\n{ocr_text_str}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)
    
    if 'pixel_values' not in inputs:
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        inputs['pixel_values'] = pixel_values

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    json_string = extract_json_from_text(result_text)

    if json_string:
        try:
            data = json5.loads(json_string)
            return filter_json_fields(data)
        except Exception as e:
            print(f"⚠️ JSON 파싱 실패: {e}")
            return {"result_text": result_text}
    else:
        print("⚠️ JSON 블록을 찾을 수 없습니다.")
        return {"result_text": result_text}

# -----------------------------
# 5. 후처리: 키 필터 + null 처리
# -----------------------------
def filter_json_fields(data):
    return {k: data.get(k, None) for k in KEY_PATTERNS.keys()}

# -----------------------------
# 6. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    try:
        from drugocr import extract_text
    except ImportError:
        def extract_text(image_path):
            return [
                '[원료약품 및 분량] 이 약 1캡슐 중',
                '●유효성분: 두타스테리드(USP)............0.5mg',
                '●첨가제(동물유래성분): 젤라틴(기원동물:소, 사용',
                '부위: 가죽)',
                '●기타첨가제: 농글리세린, 부틸히드록시톨루엔, 숙',
                '신산젤라틴, 폴리옥실40경화피마자유, 프로필렌글',
                '리콜모노라우레이트, 폴록사머124, D-소르비톨액',
                '[성상]',
                '무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐',
                '[효능·효과]',
                '양성 전립선 비대증 증상의 개선, 급성 성 요저류 위험성 감소,',
                '양성 전립선 비대증과 관련된 수술 필요성 감소, 성인 남성(만18~50세)의',
                '남성형 탈모(androgenetic alopecia)의 치료',
                '[용법·용량]',
                '이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다. 캡슐 내용물에 노출시 구강 인두점막의 자극을 초래',
                '할 수 있으므로 이 약을 씹거나 쪼개지 않고 통째로 삼켜 복용해야 한다. 이 약은 피부를 통해서',
                '흡수된다. 따라서 이 약의 흡수 가능성과 남자 태아에게 미치는 태자 기형의 위험 가능성 때문에',
                '임신했거나 임신기 가능성이 있는 여성이 이 약을 취급해서는 안된다. 또, 여성은 이 약을 취급할',
                '때마다 주의해야 하고 누출되는 캡슐 과의 접촉을 피해야 한다. 만약 캡슐이 새어 이 약과 접촉한',
                '경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다. 이하 첨부문서 참조 [저장방법] 밀폐용기,30°C이하 보관',
                '[제조의뢰자](주)제뉴원사이언스 세종특별자치시 I|전의면 산단길 245 [제조자](주)유유제약',
                '충북 제천시 바이오밸리 1로 94'
            ]
            
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # LLM JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. 키 패턴 정의 (동물유래 성분 포함)
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'·?유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]']
}

# -----------------------------
# 2. OCR 후처리
# -----------------------------
def clean_text(text):
    if not text:
        return None
    # 줄바꿈·공백 정리
    text = re.sub(r'\s+', ' ', text)
    # 제조사/주소 제거
    text = re.sub(r'\[제조의뢰자\].*?\[제조자\].*', '', text)
    # 사용상의 주의사항 제거
    text = re.sub(r'\[사용상의 주의사항\].*', '', text)
    return text.strip() if text.strip() else None

def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue
        matched = False
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, line):
                    if buffer and current_key:
                        data[current_key] = clean_text(' '.join(buffer))
                    buffer = []
                    current_key = key
                    line_clean = re.sub(pattern, '', line).strip()
                    if line_clean:
                        buffer.append(line_clean)
                    matched = True
                    break
            if matched:
                break
        if current_key and not matched:
            buffer.append(line)

    if buffer and current_key:
        data[current_key] = clean_text(' '.join(buffer))

    return data

# -----------------------------
# 3. JSON 추출 (LLM 출력)
# -----------------------------
def extract_json_from_text(text):
    try:
        start = text.find('{')
        end = text.rfind('}') + 1
        json_text = text[start:end].replace("'", '"')
        return json5.loads(json_text)
    except Exception as e:
        print(f"⚠️ JSON 파싱 실패: {e}")
        return None

# -----------------------------
# 4. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_general(ocr_texts)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "JSON key는 다음과 같습니다: "
        + ', '.join(KEY_PATTERNS.keys()) + ". "
        "값이 없으면 null로 표시하고, 광고/제조사/사용상의 주의사항 등 약 정보와 관련 없는 내용은 무시하세요. "
        "반드시 JSON 형식으로 쌍따옴표(\") 사용. 출력 외 설명 금지.\n"
        f"OCR 항목별 텍스트:\n{preprocessed_ocr}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,  # 안정적 JSON
        temperature=0
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    data = extract_json_from_text(result_text)
    if data:
        return {k: data.get(k, None) for k in KEY_PATTERNS.keys()}
    else:
        return {"result_text": result_text}

# -----------------------------
# 5. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. 키 패턴 정의 (일반화)
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'·?유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]'],
    "사용상의 주의사항": [r'\[사용상의 주의사항\]']
}

# -----------------------------
# 2. OCR 후처리 (일반화)
# -----------------------------
def preprocess_ocr_general(ocr_texts):
    data = {k: [] for k in KEY_PATTERNS.keys()}
    current_key = None

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue

        matched = False
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, line):
                    # 새로운 키 발견 시 이전 키의 버퍼를 저장하고 새 키로 전환
                    current_key = key
                    line_clean = re.sub(pattern, '', line).strip()
                    if line_clean:
                        data[current_key].append(line_clean)
                    matched = True
                    break
            if matched:
                break
        
        if not matched and current_key:
            data[current_key].append(line)

    # 리스트를 문자열로 합치고 불필요한 공백 제거
    processed_data = {}
    for k, v in data.items():
        processed_data[k] = ' '.join(v).replace('  ', ' ') if v else None
    
    return processed_data


# -----------------------------
# 3. JSON 블록 추출
# -----------------------------
def extract_json_from_text(text):
    text_clean = re.sub(r'```json|```', '', text, flags=re.IGNORECASE).strip()
    matches = re.findall(r'\{.*?\}', text_clean, flags=re.DOTALL)
    if matches:
        json_string = matches[-1].replace("'", '"').strip()
        return json_string
    else:
        return None

# -----------------------------
# 4. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_general(ocr_texts)

    ocr_text_str = json.dumps(preprocessed_ocr, ensure_ascii=False, indent=2)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "JSON key는 '원료약품 및 그 분량', '유효성분', '첨가제(보존제)', '첨가제(동물유래성분)', '기타첨가제', '성상', '효능·효과', '용법·용량', '사용상의 주의사항'입니다. "
        "값이 없으면 null로 표시하고, 광고/제조사 등 약 정보와 관련 없는 내용은 무시하세요.\n"
        f"OCR 항목별 텍스트:\n{ocr_text_str}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)
    
    if 'pixel_values' not in inputs:
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        inputs['pixel_values'] = pixel_values

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    json_string = extract_json_from_text(result_text)

    if json_string:
        try:
            data = json5.loads(json_string)
            return filter_json_fields(data)
        except Exception as e:
            print(f"⚠️ JSON 파싱 실패: {e}")
            return {"result_text": result_text}
    else:
        print("⚠️ JSON 블록을 찾을 수 없습니다.")
        return {"result_text": result_text}

# -----------------------------
# 5. 후처리: 키 필터 + null 처리
# -----------------------------
def filter_json_fields(data):
    return {k: data.get(k, None) for k in KEY_PATTERNS.keys()}

# -----------------------------
# 6. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00685.jpg"
    
    # OCR 추출
    try:
        from drugocr import extract_text
    except ImportError:
        def extract_text(image_path):
            return [
                '[원료약품 및 분량] 이 약 1캡슐 중',
                '유효성분:두타스테리드(USP)0.5mg',
                '첨가제(동물유래성분):젤라틴(기원동물:소,사용',
                '부위:가죽)',
                '기타첨가제:농글리세린,부틸히드록시톨루엔,숙',
                '신산젤라틴,폴리옥실40경화피마자유,프로필렌글',
                '리콜모노라우레이트,폴록사머124,D-소르비톨액',
                '[성상]',
                '무색투명한 내용물이 들어있는 미황색의 투명한',
                '타원형 연질캡슐',
                '[효능·효과]',
                '양성 전립선 비대증 증상의 개선,급성 성 요저류 위험성',
                '감소,양성 전립선 비대증과 관련된 수술 필요성 감소,',
                '성인 남성(만18~50세)의 남성형 탈모(androgenetic',
                'alopecia)의 치료',
                '[용법·용량]',
                '이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다.',
                '캡슐 내용물에 노출시 구강 인두점막의 자극을 초래',
                '할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로',
                '삼켜 복용해야 한다. 이 약은 식사와 관계없이 복용할',
                '수 있다.신장애 환자 또는 노인 환자에서 이 약의',
                '용량을 조절할 필요는 없다. 간장애 환자에게 이 약을',
                '투여한 자료가 없기 때문에 간장애 환자에서의 이',
                '약의 권장용량은 확립되어 있지 않다.',
                '[사용상의 주의사항]',
                '1.경고1)여성에게 노출시 남자 태아에 미치는 위험성 이 약은',
                '피부를 통 통해서 서 흡수된다.따라서 이 약의 흡수 가능성과 남자',
                '태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나',
                '임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다. 또,',
                '여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐',
                '과의 접촉을 피해야 한다. 만약 캡슐이 새어 이 약과 접촉한',
                '경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다.',
                '이하 첨부문서 참조',
                '[저장방법] 밀폐용기,30°C이하 보관',
                '[제조의뢰자](주)제뉴원사이언스',
                '세종특별자치시 I|전의면 산단길 245',
                '[제조자](주)유유제약',
                '충북 제천시 바이오밸리 1로 94'
            ]
            
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # LLM JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import re
from PIL import Image
from drugocr import extract_text

# -----------------------------
# 1. 키 패턴 정의
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]']
}

# -----------------------------
# 2. OCR 후처리
# -----------------------------
def clean_text(text):
    if not text:
        return None
    # 줄바꿈/여러 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 광고/제조사/사용상의 주의사항 제거
    text = re.sub(r'\[사용상의 주의사항\].*', '', text)
    text = re.sub(r'\[제조의뢰자\].*?\[제조자\].*', '', text)
    return text.strip() if text.strip() else None

def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue
        matched = False
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, line):
                    if buffer and current_key:
                        data[current_key] = clean_text(' '.join(buffer))
                    buffer = []
                    current_key = key
                    line_clean = re.sub(pattern, '', line).strip()
                    if line_clean:
                        buffer.append(line_clean)
                    matched = True
                    break
            if matched:
                break
        if current_key and not matched:
            buffer.append(line)

    if buffer and current_key:
        data[current_key] = clean_text(' '.join(buffer))

    return data

# -----------------------------
# 3. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # OCR 후처리 → JSON
    json_result = preprocess_ocr_general(ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import re
from PIL import Image
from drugocr import extract_text

# -----------------------------
# 1. 키 패턴 정의
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능·효과\]'],
    "용법·용량": [r'\[용법·용량\]']
}

# -----------------------------
# 2. OCR 후처리
# -----------------------------
def clean_text(text):
    if not text:
        return None
    # 줄바꿈/여러 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 광고/제조사/사용상의 주의사항 제거
    text = re.sub(r'\[사용상의 주의사항\].*', '', text)
    text = re.sub(r'\[제조의뢰자\].*?\[제조자\].*', '', text)
    return text.strip() if text.strip() else None

def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue
        matched = False
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, line):
                    if buffer and current_key:
                        data[current_key] = clean_text(' '.join(buffer))
                    buffer = []
                    current_key = key
                    line_clean = re.sub(pattern, '', line).strip()
                    if line_clean:
                        buffer.append(line_clean)
                    matched = True
                    break
            if matched:
                break
        if current_key and not matched:
            buffer.append(line)

    if buffer and current_key:
        data[current_key] = clean_text(' '.join(buffer))

    return data

# -----------------------------
# 3. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00298.jpg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # OCR 후처리 → JSON
    json_result = preprocess_ocr_general(ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. 키 패턴 정의 (일반화)
# -----------------------------
KEY_PATTERNS = {
    "원료약품 및 그 분량": [r'\[원료약품.*\]'],
    "유효성분": [r'·?유효성분:'],
    "첨가제(보존제)": [r'첨가제\(보존제\):'],
    "첨가제(동물유래성분)": [r'첨가제\(동물유래성분\):'],
    "기타첨가제": [r'기타 첨가제:'],
    "성상": [r'\[성상\]'],
    "효능·효과": [r'\[효능.*효과\]'],
    "용법·용량": [r'\[용법.*용량\]'],
    "사용상의 주의사항": [r'\[사용상의 주의사항\]']
}

# -----------------------------
# 2. OCR 후처리 (일반화)
# -----------------------------
def preprocess_ocr_general(ocr_texts):
    data = {k: None for k in KEY_PATTERNS.keys()}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue
        
        # 키 패턴을 찾아내고 정렬
        found_keys = []
        for key, patterns in KEY_PATTERNS.items():
            for pattern in patterns:
                match = re.search(pattern, line)
                if match:
                    found_keys.append((match.start(), key, pattern))
        found_keys.sort()
        
        # 키 패턴을 기준으로 텍스트 분리
        if found_keys:
            if current_key and buffer:
                data[current_key] = ' '.join(buffer).replace('  ', ' ')
            
            last_idx = 0
            for start_idx, key, pattern in found_keys:
                # 이전 키의 값 저장
                if last_idx < start_idx and current_key:
                    buffer.append(line[last_idx:start_idx].strip())
                    
                current_key = key
                buffer = []
                last_idx = re.search(pattern, line).end()
                
            line_clean = line[last_idx:].strip()
            if line_clean:
                buffer.append(line_clean)
        
        elif current_key:
            buffer.append(line)
    
    if current_key and buffer:
        data[current_key] = ' '.join(buffer).replace('  ', ' ')

    # 모든 키에 대해 값이 있는지 확인
    processed_data = {k: v for k, v in data.items() if v is not None}
    
    # 예외 처리: '유효성분'과 '기타 첨가제'가 합쳐져 있는 경우
    if '유효성분' in processed_data and '기타첨가제' not in processed_data:
        match = re.search(r'(기타\s?첨가제:.*)', processed_data['유효성분'])
        if match:
            part_to_move = match.group(1)
            processed_data['기타첨가제'] = re.sub(r'기타\s?첨가제:', '', part_to_move).strip()
            processed_data['유효성분'] = processed_data['유효성분'].replace(part_to_move, '').strip()

    return processed_data


# -----------------------------
# 3. JSON 블록 추출
# -----------------------------
def extract_json_from_text(text):
    text_clean = re.sub(r'```json|```', '', text, flags=re.IGNORECASE).strip()
    matches = re.findall(r'\{.*?\}', text_clean, flags=re.DOTALL)
    if matches:
        json_string = matches[-1].replace("'", '"').strip()
        return json_string
    else:
        return None

# -----------------------------
# 4. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_general(ocr_texts)

    ocr_text_str = json.dumps(preprocessed_ocr, ensure_ascii=False, indent=2)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "JSON key는 '원료약품 및 그 분량', '유효성분', '첨가제(보존제)', '첨가제(동물유래성분)', '기타첨가제', '성상', '효능·효과', '용법·용량', '사용상의 주의사항'입니다. "
        "값이 없으면 null로 표시하고, 광고/제조사 등 약 정보와 관련 없는 내용은 무시하세요.\n"
        f"OCR 항목별 텍스트:\n{ocr_text_str}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)
    
    if 'pixel_values' not in inputs:
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        inputs['pixel_values'] = pixel_values

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    json_string = extract_json_from_text(result_text)

    if json_string:
        try:
            data = json5.loads(json_string)
            return filter_json_fields(data)
        except Exception as e:
            print(f"⚠️ JSON 파싱 실패: {e}")
            return {"result_text": result_text}
    else:
        print("⚠️ JSON 블록을 찾을 수 없습니다.")
        return {"result_text": result_text}

# -----------------------------
# 5. 후처리: 키 필터 + null 처리
# -----------------------------
def filter_json_fields(data):
    return {k: data.get(k, None) for k in KEY_PATTERNS.keys()}

# -----------------------------
# 6. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00298.jpg"
    
    # OCR 추출
    try:
        from drugocr import extract_text
    except ImportError:
        def extract_text(image_path):
            return ['케이 캡정', '[원료약품 및그분량]이약1정(206mg) 중', '유효성분:테고프라잔(별규)', '기타 첨가제:D-만니톨', '히드록시프로필셀룰로오스.', '오파드라이1분홍색(85F240134)', '장방혀', '[성상]연한 분홍색의', 'O00', '[효능 효과]1.미란성', '2.비미란성', '3.위궤양의 치료', '4.소화성 궤양', '박터파일로리', '[용법·용량]이 약은 성인에게', '1.미란성위식도역류질환의 치료:1일1회', '식도염이 치료되지 않거나', '21 .비미란성', '위궤양의 치료:1일1회1회 50mg을 8주간 경구투여한다.', '위한 항생제', '받아야 한다.', '1일2회 7일간 경구투여한다', '이약은 식사와 관계없이 투여할 수 있다.', '[사용상의 주의사항] 첨부문서 참조', "[저장방법]기밀용기,실온(1~30'C)보관", 'R', 'K-CAB', 'Tegoprazan 50mg', '50.0mg', '미결정셀룰로오스, 크로스카르멜로오스나트륨.', '콜로이드성이산화규소. 스테아르산마그네슘.', '필름코팅정', '위식도역류질환의 치료', '위식도역류질환의치료', '및/또는 만성 위축성 위염 환자에서의 헬리코', '제균을 위한 항생제 벼요요번', '이능프급', '다음과 같이 투여한다', '1회 50mg을 4주간 경구투여한다.', '증상이 계속되는 환자의 경우4주 더투여한다.', '위식도역류질환의 치료:1일 1회,1회 50mg을 4주간 경구투여한다.', '4.소화성 궤양 및/또는 만성 위축성 위염 환자에서의 헬리코박터파일로리 제균을', '병용요법:헬리코박터파일로리 감염환자들은 제균요법으로 치료', '이약50mg과 아목시실린1g.클래리트로마이신500 mg을']
            
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # LLM JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")

In [ ]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 0. 모델 불러오기
# -----------------------------
MODEL_REPO = "NCSOFT/VARCO-VISION-2.0-1.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# -----------------------------
# 1. OCR 후처리 (범용 버전)
# -----------------------------
def preprocess_ocr_global(ocr_texts):
    data = {}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue
        
        # 동적으로 키 패턴을 찾음 (예: [원료약품 및 그 분량] 또는 유효성분:)
        match = re.match(r'\[(.*?)\]|\s*([^:]+):', line)
        
        if match:
            # 이전 키의 값 저장
            if current_key and buffer:
                data[current_key] = ' '.join(buffer).replace('  ', ' ')
            
            # 새 키 추출 및 버퍼 초기화
            if match.group(1):
                current_key = match.group(1).strip()
                remaining_text = line[match.end(1) + 1:].strip()
            else:
                current_key = match.group(2).strip()
                remaining_text = line[match.end(2) + 1:].strip()
            
            buffer = []
            if remaining_text:
                buffer.append(remaining_text)
        
        elif current_key:
            # 현재 키에 대한 값 추가
            buffer.append(line)
    
    # 마지막 키의 값 저장
    if current_key and buffer:
        data[current_key] = ' '.join(buffer).replace('  ', ' ')
        
    return data


# -----------------------------
# 2. JSON 블록 추출
# -----------------------------
def extract_json_from_text(text):
    text_clean = re.sub(r'```json|```', '', text, flags=re.IGNORECASE).strip()
    matches = re.findall(r'\{.*?\}', text_clean, flags=re.DOTALL)
    if matches:
        json_string = matches[-1].replace("'", '"').strip()
        return json_string
    else:
        return None

# -----------------------------
# 3. LLM JSON 변환
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    preprocessed_ocr = preprocess_ocr_global(ocr_texts)

    ocr_text_str = json.dumps(preprocessed_ocr, ensure_ascii=False, indent=2)

    prompt = (
        "<image>\n"
        "아래 OCR 텍스트를 참고하여 약 정보만 JSON으로 정리하세요. "
        "광고/제조사 등 약 정보와 관련 없는 내용은 무시하고, 모든 정보를 JSON에 포함하세요.\n"
        f"OCR 항목별 텍스트:\n{ocr_text_str}\n"
    )

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(device)
    
    if 'pixel_values' not in inputs:
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        inputs['pixel_values'] = pixel_values

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )

    result_text = processor.decode(outputs[0], skip_special_tokens=True)
    json_string = extract_json_from_text(result_text)

    if json_string:
        try:
            data = json5.loads(json_string)
            return data
        except Exception as e:
            print(f"⚠️ JSON 파싱 실패: {e}")
            return {"result_text": result_text}
    else:
        print("⚠️ JSON 블록을 찾을 수 없습니다.")
        return {"result_text": result_text}

# -----------------------------
# 4. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00298.jpg"
    
    # OCR 추출
    try:
        from drugocr import extract_text
    except ImportError:
        def extract_text(image_path):
            return ['케이 캡정', '[원료약품 및그분량]이약1정(206mg) 중', '유효성분:테고프라잔(별규)', '기타 첨가제:D-만니톨', '히드록시프로필셀룰로오스.', '오파드라이1분홍색(85F240134)', '장방혀', '[성상]연한 분홍색의', 'O00', '[효능 효과]1.미란성', '2.비미란성', '3.위궤양의 치료', '4.소화성 궤양', '박터파일로리', '[용법·용량]이 약은 성인에게', '1.미란성위식도역류질환의 치료:1일1회', '식도염이 치료되지 않거나', '21 .비미란성', '위궤양의 치료:1일1회1회 50mg을 8주간 경구투여한다.', '위한 항생제', '받아야 한다.', '1일2회 7일간 경구투여한다', '이약은 식사와 관계없이 투여할 수 있다.', '[사용상의 주의사항] 첨부문서 참조', "[저장방법]기밀용기,실온(1~30'C)보관", 'R', 'K-CAB', 'Tegoprazan 50mg', '50.0mg', '미결정셀룰로오스, 크로스카르멜로오스나트륨.', '콜로이드성이산화규소. 스테아르산마그네슘.', '필름코팅정', '위식도역류질환의 치료', '위식도역류질환의치료', '및/또는 만성 위축성 위염 환자에서의 헬리코', '제균을 위한 항생제 벼요요번', '이능프급', '다음과 같이 투여한다', '1회 50mg을 4주간 경구투여한다.', '증상이 계속되는 환자의 경우4주 더투여한다.', '위식도역류질환의 치료:1일 1회,1회 50mg을 4주간 경구투여한다.', '4.소화성 궤양 및/또는 만성 위축성 위염 환자에서의 헬리코박터파일로리 제균을', '병용요법:헬리코박터파일로리 감염환자들은 제균요법으로 치료', '이약50mg과 아목시실린1g.클래리트로마이신500 mg을']
            
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # LLM JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import re
from PIL import Image
from drugocr import extract_text

# -----------------------------
# 1. OCR 후처리
# -----------------------------
def clean_text(text):
    if not text:
        return None
    # 줄바꿈/여러 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 광고/제조사/사용상의 주의사항 제거
    text = re.sub(r'\[사용상의 주의사항\].*', '', text)
    text = re.sub(r'\[제조의뢰자\].*?\[제조자\].*', '', text)
    return text.strip() if text.strip() else None

def extract_key_value(line):
    # 대괄호 [] 형식
    match_bracket = re.match(r'\[(.*?)\]\s*(.*)', line)
    if match_bracket:
        key, value = match_bracket.groups()
        return key.strip(), value.strip() if value else None
    # 콜론 : 형식
    match_colon = re.match(r'([^:]+):\s*(.*)', line)
    if match_colon:
        key, value = match_colon.groups()
        return key.strip(), value.strip() if value else None
    return None, line.strip()

def preprocess_ocr_global(ocr_texts):
    data = {}
    current_key = None
    buffer = []

    for line in ocr_texts:
        line = line.strip()
        if not line:
            continue

        key, value = extract_key_value(line)

        if key:
            # 이전 buffer 저장
            if current_key and buffer:
                data[current_key] = clean_text(' '.join(buffer))
            current_key = key
            buffer = [value] if value else []
        else:
            if current_key:
                buffer.append(value)

    if current_key and buffer:
        data[current_key] = clean_text(' '.join(buffer))

    # value 없는 key는 null 처리
    for k in data.keys():
        if data[k] is None:
            data[k] = None

    return data

# -----------------------------
# 2. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # OCR 후처리 → JSON
    json_result = preprocess_ocr_global(ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import re
from drugocr import extract_text

# -----------------------------
# 1. OCR 후처리 함수
# -----------------------------
def clean_text(text):
    """줄바꿈, 여러 공백 제거 및 불필요한 내용 제거"""
    if not text:
        return None
    # 줄바꿈, 여러 공백을 하나의 공백으로 통일
    text = re.sub(r'\s+', ' ', text)
    # 불필요한 항목 제거
    text = re.sub(r'\[사용상의 주의사항\].*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[제조의뢰자\].*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[제조자\].*', '', text, flags=re.IGNORECASE)
    return text.strip() if text.strip() else None

def extract_key_value(line):
    """한 줄에서 key와 value 추출"""
    line = line.strip()
    if not line:
        return None, None

    # [key] value 형식
    match_bracket = re.match(r'\[(.*?)\]\s*(.*)', line)
    if match_bracket:
        key, value = match_bracket.groups()
        return key.strip(), value.strip() if value else None

    # key: value 형식
    match_colon = re.match(r'([^:]+):\s*(.*)', line)
    if match_colon:
        key, value = match_colon.groups()
        return key.strip(), value.strip() if value else None

    return None, line.strip()

def preprocess_ocr_global(ocr_texts):
    """OCR 텍스트 리스트를 key-value JSON으로 변환"""
    data = {}
    current_key = None
    buffer = []

    for line in ocr_texts:
        key, value = extract_key_value(line)

        if key:  # 새로운 key 발견
            # 이전 buffer 저장
            if current_key and buffer:
                combined = ' '.join(filter(None, buffer))
                cleaned_value = clean_text(combined)
                if cleaned_value:
                    data[current_key] = cleaned_value
            current_key = key
            buffer = [value] if value else []
        else:
            if current_key:
                buffer.append(value)

    # 마지막 buffer 저장
    if current_key and buffer:
        combined = ' '.join(filter(None, buffer))
        cleaned_value = clean_text(combined)
        if cleaned_value:
            data[current_key] = cleaned_value

    return data

# -----------------------------
# 2. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00806.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # JSON 변환
    json_result = preprocess_ocr_global(ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        # ensure_ascii=False 유지, indent=2로 읽기 쉽게 저장
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")


In [ ]:
import os
import json
import re
from drugocr import extract_text

# -----------------------------
# 1. OCR 후처리 함수
# -----------------------------
def clean_and_correct_text(text):
    """줄바꿈, 여러 공백 제거 및 띄어쓰기 교정"""
    if not text:
        return None
    # 줄바꿈, 여러 공백을 하나의 공백으로 통일
    text = re.sub(r'\s+', ' ', text)
    
    # 특정 띄어쓰기 오류 수동 교정 (필요한 경우)
    text = text.replace('숙 신산', '숙신산')
    text = text.replace('프로필렌글 리콜', '프로필렌글리콜')
    text = text.replace('기타첨가제:농글리세린', '농글리세린')
    
    return text.strip() if text.strip() else None

def extract_key_value(line):
    """한 줄에서 key와 value 추출"""
    line = line.strip()
    if not line:
        return None, None

    # [key] value 형식
    match_bracket = re.match(r'\[(.*?)\]\s*(.*)', line)
    if match_bracket:
        key, value = match_bracket.groups()
        return key.strip(), value.strip() if value else None

    # key: value 형식
    match_colon = re.match(r'([^:]+):\s*(.*)', line)
    if match_colon:
        key, value = match_colon.groups()
        return key.strip(), value.strip() if value else None

    return None, line.strip()

def preprocess_ocr_global(ocr_texts):
    """OCR 텍스트 리스트를 key-value JSON으로 변환"""
    data = {}
    current_key = None
    buffer = []
    
    # 약과 관련 없는 항목 키 리스트
    exclude_keys = ['저장방법', '제조의뢰자', '제조자', '사용상의 주의사항']

    for line in ocr_texts:
        key, value = extract_key_value(line)

        # '첨가제(동물유래성분)'와 '부위'를 하나로 병합
        if key == '부위':
            if '첨가제(동물유래성분)' in data:
                data['첨가제(동물유래성분)'] += ' ' + value
            continue # 다음 라인으로

        if key:  # 새로운 key 발견
            if current_key and buffer and current_key not in exclude_keys:
                combined = ' '.join(filter(None, buffer))
                cleaned_value = clean_and_correct_text(combined)
                if cleaned_value:
                    data[current_key] = cleaned_value
            current_key = key
            buffer = [value] if value else []
        else:
            if current_key:
                buffer.append(value)

    # 마지막 buffer 저장
    if current_key and buffer and current_key not in exclude_keys:
        combined = ' '.join(filter(None, buffer))
        cleaned_value = clean_and_correct_text(combined)
        if cleaned_value:
            data[current_key] = cleaned_value

    return data

# -----------------------------
# 2. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"/home/wanted-1/PotenupWorkspace/sep-project1/nanhye/__pycache__/data/medicine_00451.jpeg"
    
    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)
    
    # JSON 변환
    json_result = preprocess_ocr_global(ocr_texts)
    
    # JSON 예쁘게 출력
    print("📌 최종 JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))
    
    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료: {output_file}")